In [ ]:
# ============================================================
#  Neural Network Surrogate for Black-Box Optimisation (BBO)
# ============================================================
#  Overview:
#  This code trains a PyTorch neural network surrogate model to
#  approximate a black-box function and identify the next best
#  query point via Monte Carlo sampling.
#
#  Core idea:
#  - The surrogate learns the relationship between inputs (X)
#    and outputs (y).
#  - Monte Carlo sampling generates many random candidates.
#  - The surrogate predicts their outputs.
#  - The best candidate (highest predicted y) is chosen as the
#    next query point.
#
#  Monte Carlo approach:
#  ---------------------
#  Since black-box functions are unknown and costly to query,
#  Monte Carlo sampling provides an efficient, probabilistic
#  method to explore the input space and estimate promising
#  regions for improvement.
#
#  Exploration vs. Exploitation Controls:
#  --------------------------------------
#  - Learning Rate (`lr`):
#       ↑ Higher → faster, noisier updates (more exploration)
#       ↓ Lower  → slower, stable convergence (more exploitation)
#  - Batch Size (`batch_size`):
#       ↓ Smaller → more stochastic gradient (exploration)
#       ↑ Larger  → smoother gradient, focus on known good regions (exploitation)
#  - Monte Carlo Samples (`n_candidates`):
#       ↑ More → broader coverage of search space (exploration)
#       ↓ Fewer → faster computation, focus on known regions (exploitation)
#
# ============================================================

import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
from sklearn.preprocessing import StandardScaler
import numpy as np

# ------------------------------------------------------------
# Set random seeds for reproducibility
# ------------------------------------------------------------
torch.manual_seed(42)
np.random.seed(42)

# ------------------------------------------------------------
# Define Neural Network Surrogate Model
# ------------------------------------------------------------
class SurrogateNN(nn.Module):
    def __init__(self, input_dim):
        super(SurrogateNN, self).__init__()
        self.net = nn.Sequential(
            nn.Linear(input_dim, 64),
            nn.ReLU(),
            nn.Linear(64, 32),
            nn.ReLU(),
            nn.Linear(32, 1)
        )

    def forward(self, x):
        return self.net(x)

# ------------------------------------------------------------
# Training Function with Optional Mini-Batching
# ------------------------------------------------------------
def train_surrogate(X_train, y_train, input_dim, lr=0.01, epochs=300, batch_size=None):
    """
    Trains the neural network surrogate.
    If batch_size is None, uses full-batch training.
    """
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model = SurrogateNN(input_dim).to(device)
    optimizer = optim.Adam(model.parameters(), lr=lr)
    loss_fn = nn.MSELoss()

    X_t = torch.tensor(X_train, dtype=torch.float32).to(device)
    y_t = torch.tensor(y_train, dtype=torch.float32).view(-1, 1).to(device)

    if batch_size is not None:
        dataset = TensorDataset(X_t, y_t)
        loader = DataLoader(dataset, batch_size=batch_size, shuffle=True)
    else:
        loader = [(X_t, y_t)]  # full-batch fallback

    for epoch in range(epochs):
        for xb, yb in loader:
            optimizer.zero_grad()
            preds = model(xb)
            loss = loss_fn(preds, yb)
            loss.backward()
            optimizer.step()

    return model

# ------------------------------------------------------------
# Monte Carlo Sampling to Find Next Query Point
# ------------------------------------------------------------
def suggest_next_query(model, input_dim, n_candidates=10000):
    """
    Uses Monte Carlo sampling to suggest the next query point.
    """
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model.eval()

    # Generate random candidate points uniformly in [0,1]^d
    candidates = np.random.rand(n_candidates, input_dim)
    with torch.no_grad():
        preds = model(torch.tensor(candidates, dtype=torch.float32).to(device)).cpu().numpy()

    # Select candidate with highest predicted value
    best_idx = np.argmax(preds)
    x_next = candidates[best_idx]

    return x_next, preds[best_idx]


# ------------------------------------------------------------
# Denormalisation Helper Function
# ------------------------------------------------------------
def denormalize_results(x_next_norm, pred_y_norm):
    """
    Convert the normalised next query and predicted output back
    to their original scales.
    """
    # Recover X
    x_next_real = x_scaler.inverse_transform(x_next_norm.reshape(1, -1)).flatten()

    # Recover y
    y_pred_real = pred_y_norm * y_std + y_mean

    return x_next_real, y_pred_real


# ------------------------------------------------------------
# Example Usage
# ------------------------------------------------------------
# Assume you have:
#   X: 2D numpy array of inputs (n_samples x n_features)
#   y: 1D numpy array of corresponding outputs

# Example placeholders (replace with real data)
# X = np.load("initial_inputs.npy")
# y = np.load("initial_outputs.npy")
#from google.colab import drive
#drive.mount('/content/drive')

function = '8'  ## Enter the function number being processed as a string.
path = 'drive/MyDrive/data/' # Enter path to your initial files
given_X = path + 'fn' + function + '_initial_inputs.npy'
given_y = path + 'fn' + function + '_initial_outputs.npy'


# ------------------------------------------------------------
# Load initial data
# ------------------------------------------------------------
X = np.load(given_X)        # shape (n0, d)
y = np.load(given_y)       # shape (n0,)

# ------------------------------------------------------------
# Append new information
# ------------------------------------------------------------
if function == '1':
  X = np.append(X,[[0.701438, 0.897524]], axis=0)  # Append week1 inputs
  y = np.append(y, 4.907008577514035e-55)         # Append week1 outputs
  X = np.append(X,[[0.608788, 0.208839]], axis=0)  # Append week2 inputs
  y = np.append(y, -3.407789887257921e-57)         # Append week2 outputs
  X = np.append(X,[[0.937387, 0.537998]], axis=0)  # Append week3 inputs
  y = np.append(y, -1.8248983945966781e-72)         # Append week3 outputs
  X = np.append(X,[[0.401144, 0.347316]], axis=0)  # Append week4 inputs
  y = np.append(y, -0.000052943333535254514)         # Append week4 outputs
elif function == '2':
  X = np.append(X,[[0.695813, 0.000000]], axis=0)  # Append week inputs
  y = np.append(y, 0.6257440183692108)         # Append week1 outputs
  X = np.append(X,[[0.969061, 0.774664]], axis=0)  # Append week2 inputs
  y = np.append(y, 0.033917749094575816)         # Append week2 outputs
  X = np.append(X,[[0.690739, 0.962975]], axis=0)  # Append week3 inputs
  y = np.append(y, 0.6440194651539941)         # Append week3 outputs
  X = np.append(X,[[0.000000, 0.982469]], axis=0)  # Append week4 inputs
  y = np.append(y, 0.011920662637457252)         # Append week4 outputs
elif function == '3':
  X = np.append(X,[[0.000000, 0.000000, 0.000000]], axis=0)  # Append week1 inputs
  y = np.append(y, -0.1755204613669372)         # Append week1 outputs
  X = np.append(X,[[0.999999, 0.183024, 0.788109]], axis=0)  # Append week2 inputs
  y = np.append(y, -0.10594389054163186)         # Append week2 outputs
  X = np.append(X,[[0.937840, 0.999999, 0.405928]], axis=0)  # Append week3 inputs
  y = np.append(y, -0.05889151139553504)         # Append week3 outputs
  X = np.append(X,[[0.523777, 0.465195, 0.442681]], axis=0)  # Append week4 inputs
  y = np.append(y, -0.005996811424753983)         # Append week4 outputs
elif function == '4':
  X = np.append(X,[[0.440415, 0.425453, 0.378353, 0.397106]], axis=0)  # Append week1 inputs
  y = np.append(y, 0.2601025838576061)         # Append week1 outputs
  X = np.append(X,[[0.410011, 0.415255, 0.344454, 0.438808]], axis=0)  # Append week2 inputs
  y = np.append(y, 0.3983591561199096)         # Append week2 outputs
  X = np.append(X,[[0.384038, 0.412994, 0.399499, 0.436890]], axis=0)  # Append week3 inputs
  y = np.append(y, 0.2987152140170406)         # Append week3 outputs
  X = np.append(X,[[0.391529, 0.451226, 0.356835, 0.420242]], axis=0)  # Append week4 inputs
  y = np.append(y, 0.06384840143364956)         # Append week4 outputs
elif function == '5':
  X = np.append(X,[[0.000000, 0.827185, 0.999999, 0.999999]], axis=0)  # Append week1 inputs
  y = np.append(y, 2781.638812419282)         # Append week1 outputs
  X = np.append(X,[[0.058940, 0.195873, 0.872581, 0.998561]], axis=0)  # Append week2 inputs
  y = np.append(y, 848.8896402098709)         # Append week2 outputs
  X = np.append(X,[[0.166206, 0.999999, 0.999999, 0.999999]], axis=0)  # Append week3 inputs
  y = np.append(y, 4444.256290210307)         # Append week3 outputs
  X = np.append(X,[[0.259704, 0.866240, 0.999999, 0.078304]], axis=0)  # Append week4 inputs
  y = np.append(y, 837.6996715082641)         # Append week4 outputs
elif function == '6':
  X = np.append(X,[[0.464910, 0.242338, 0.574752, 0.999999, 0.000000]], axis=0)  # Append week1 inputs
  y = np.append(y, -0.5265043497038704)         # Append week1 outputs
  X = np.append(X,[[0.537065, 0.317653, 0.626041, 0.898937, 0.148218]], axis=0)  # Append week2 inputs
  y = np.append(y, -0.41510731628352715)         # Append week2 outputs
  X = np.append(X,[[0.455585, 0.174069, 0.999999, 0.999999, 0.292903]], axis=0)  # Append week3 inputs
  y = np.append(y, -1.0104998624615082)         # Append week3 outputs
  X = np.append(X,[[0.525055, 0.369255, 0.490228, 0.765387, 0.025091]], axis=0)  # Append week4 inputs
  y = np.append(y, -0.3481623700173726)         # Append week4 outputs
elif function == '7':
  X = np.append(X,[[0.000000, 0.247036, 0.408965, 0.217149, 0.377534, 0.746590]], axis=0)  # Append week1 inputs
  y = np.append(y, 2.3034568430941222)         # Append week1 outputs
  X = np.append(X,[[0.000000, 0.181841, 0.435826, 0.062982, 0.361647, 0.858489]], axis=0)  # Append week2 inputs
  y = np.append(y, 1.3279380639726712)         # Append week2 outputs
  X = np.append(X,[[0.000000, 0.166738, 0.211207, 0.080984, 0.381370, 0.746608]], axis=0)  # Append week3 inputs
  y = np.append(y, 1.433484749556216)         # Append week3 outputs
  X = np.append(X,[[0.000000, 0.198493, 0.750365, 0.247059, 0.370826, 0.980737]], axis=0)  # Append week4 inputs
  y = np.append(y, 1.239424289285777)         # Append week4 outputs
elif function == '8':
  X = np.append(X,[[0.060275, 0.000000, 0.134973, 0.000000, 0.999999, 0.404343, 0.057755, 0.516689]], axis=0)  # Append week1 inputs
  y = np.append(y, 9.8814582425914)         # Append week1 outputs
  X = np.append(X,[[0.122654, 0.153991, 0.162413, 0.045600, 0.999999, 0.536650, 0.260832, 0.932950]], axis=0)  # Append week2 inputs
  y = np.append(y, 9.9450768395815)         # Append week2 outputs
  X = np.append(X,[[0.085442, 0.318585, 0.000000, 0.239064, 0.999999, 0.927570, 0.142651, 0.932950]], axis=0)  # Append week3 inputs
  y = np.append(y, 9.6920435401985)         # Append week3 outputs
  X = np.append(X,[[0.266427, 0.000000, 0.185600, 0.000000, 0.999999, 0.210401, 0.179882, 0.156373]], axis=0)  # Append week4 inputs
  y = np.append(y, 9.7659726871796)         # Append week4 outputs



# ------------------------------------------------------------
# Normalise X and y
# ------------------------------------------------------------
# After loading your data (e.g. X, y = np.load(...))
x_scaler = StandardScaler()
Xn = x_scaler.fit_transform(X)

y_mean = y.mean()
y_std = y.std() if y.std() > 0 else 1.0
yn = (y - y_mean) / y_std   # Normalize target manually (avoid sklearn overhead)


input_dim = Xn.shape[1]
batch_size = 8  # comment this line to use full-batch training

# Train surrogate model
model = train_surrogate(Xn, yn, input_dim=input_dim, lr=0.01, epochs=500, batch_size=batch_size)

# Suggest next query point using Monte Carlo search
x_next_norm, y_pred_norm = suggest_next_query(model, input_dim=input_dim, n_candidates=20000)

# ------------------------------------------------------------
# Convert back to original scales
# ------------------------------------------------------------
x_next, y_pred = denormalize_results(x_next_norm, y_pred_norm)

print("Suggested next input (x_next):", '-'.join(map(str, np.round(x_next, 6))))
print("Predicted output estimate:", float(y_pred[0]))


Suggested next input (x_next): 0.514029-0.459555-0.534367-0.406509-0.74451-0.66331-0.578111-0.756422
Predicted output estimate: 8.434964868576083
